In [1]:
#library들 import
#bs4, selenium을 메인으로
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [7]:
browser = webdriver.Chrome('/Users/hoon2hooni/usr/bin/chromedriver') #크롬 드라이버 입력
browser.implicitly_wait(5)

In [8]:
url = "https://travel.interpark.com/checkinnow/categoryMain/search?type=hotel&regioncode1=42001"
browser.get(url)

In [4]:
#browser(Chrome driver) -> return ending_sign, 각 리뷰 데이터 배열들

def get_reviews(browser): 
    ReviewText_arr = []
    ReviewRating_arr = []
    ReviewDate_arr = []
    ReviewTitle_arr = []
    
    #ending_sign의 경우 평점이 없는 리뷰들을 추출할때 추출을 끝내도록 해주는sign  
    ending_sign = False
    i = 3
    
    #items 리뷰 갯수 파악 해주는 변수
    items = browser.find_elements_by_xpath("/html/body/section/div/section/div[1]/div[2]/div")
    #items 내부에 2개의 리뷰와 관련 없는 내용 있음
    item_numbers=len(items[2:])
    
    while i < 3 + item_numbers:
        try :
            print("시작", i-2)
            #ReviewRating css selector로 접근해서 text method로 데이터 추출, 전처리(일자제거, . -> - 로 바꿔줌)
            ReviewRating = browser.find_element_by_css_selector("#review > div.reviewsSearchWrap > div:nth-child({}) > div.infoWrap > p.scoreNum".format(i)).text
            #ReviewDate css selector로 접근해서 text method로 데이터 추출, 전처리(일자제거, . -> - 로 바꿔줌)
            ReviewDate = browser.find_element_by_css_selector("#review > div.reviewsSearchWrap > div:nth-child({}) > div.infoWrap > p.txt > span:nth-child(2)".format(i)).text.replace(".","-")[:7]
            #ReviewText, ReviewRating, ReviewTitle css selector로 접근 후 text method로 데이터 추출
            ReviewText = browser.find_element_by_css_selector("#review > div.reviewsSearchWrap > div:nth-child({}) > div.conWrap > p".format(i)).text
            ReviewTitle = browser.find_element_by_css_selector("#review > div.reviewsSearchWrap > div:nth-child({}) > div.titWrap > p".format(i)).text

            #각 배열에 저장 
            ReviewText_arr.append(ReviewText) 
            ReviewRating_arr.append(ReviewRating)
            ReviewDate_arr.append(ReviewDate)
            ReviewTitle_arr.append(ReviewTitle)     
            i += 1  
        except:
            
            #평점이 없을 경우 RevieRating를 뽑을때 에러발행
            ending_sign = True
            print("끝")
            break
    return [ending_sign, ReviewDate_arr, ReviewRating_arr, ReviewTitle_arr ,ReviewText_arr]
    

In [28]:
#한 호텔 내에서 리뷰 접근하기
def page_search(browser):
    print("들어왔다")
    time.sleep(5)
    date_arr = []
    rating_arr = []
    title_arr = []
    text_arr = []
    #review들중 평점이 없는 review들이 발견되었을때 더이상 추출하지 않기 위해 찾아주는 지표
    end_by_reviews = False
    one_hotel_reviews =[date_arr, rating_arr, title_arr, text_arr] 
    while True:
        #호텔 다음페이지로 넘어가게 하는 반복
        try :
            idx = 1
            print("hotel 시작")
            time.sleep(2)
            #이 페이지가 마지막 페이지인지 알려주는 지표
            remember =browser.find_element_by_css_selector("#layout-popup > section > div > section > div.pageNumBox > ul > li").text
            #button 갯수
            count_buttons = len(browser.find_element_by_css_selector("#layout-popup > section > div > section > div.pageNumBox > ul").text.split('\n'))
            while idx <= count_buttons: 
                #버튼수만큼 반복
                print(f'{idx}버튼 시작')
                try:
                    browser.find_element_by_css_selector('div.pageNumBox > ul > li:nth-child({})'.format(idx)).click()
                    time.sleep(2)
                    reviews_arr = get_reviews(browser)
                    idx += 1
                    if reviews_arr[0]:
                        end_by_reviews = True
                    #배열 채워주기
                    for i in range(1,5):
                        one_hotel_reviews[i-1]+=reviews_arr[i]
                    if end_by_reviews == True:
                        #평점 없는 리뷰발견되 멈추었을경우 이대로 바로 리턴해줌
                        return one_hotel_reviews
                except:
                    break
            #다음 버튼 누르기        
            browser.find_element_by_css_selector("#layout-popup > section > div > section > div.pageNumBox > button.nextBtn").click()
            time.sleep(2)
            #마지막페이지일경우 버튼의 번호 변경이 없음 이를통해 마지막페이지인지 확인.
            if remember == browser.find_element_by_css_selector("#layout-popup > section > div > section > div.pageNumBox > ul > li").text:
                break
        except:
            #오류일때 더이상 반복안하고 멈추어줌
            print("오류발견")
            break
            
    return one_hotel_reviews

#첫페이지가 마지막페이지일대 그리고 마지막페이지일때 구현해주어야함..ㅜ





In [10]:
#호텔 url접근하기
def go_inside_url(browser):
    time.sleep(4)
    url_idx = 1
    one_page_hotel_reviews=[[] for i in range(7)]
    while True:
        try:
            HotelName = browser.find_element_by_css_selector("ul > li:nth-child({}) > div.col.data > div:nth-child(1) > div.titArea > a > strong".format(url_idx)).text
            HotelAddress = browser.find_element_by_css_selector("ul > li:nth-child({}) > div.col.data > div:nth-child(2) > div > span:nth-child(1)".format(url_idx)).text[:-5]
            HotelRating = browser.find_element_by_css_selector(" ul > li:nth-child({}) > div.col.data > div:nth-child(3) > div.ratingArea > span.score".format(url_idx)).text[3:]
            hotel_info =[HotelName, HotelAddress, HotelRating]
            new_url = browser.find_element_by_css_selector("ul > li:nth-child({}) > div.col.data > div:nth-child(1) > div.titArea > a".format(url_idx)).get_attribute("href")
            review_browser = webdriver.Chrome('/Users/hoon2hooni/usr/bin/chromedriver')
            review_browser.implicitly_wait(5)
            review_browser.get(new_url)
            reviews = page_search(review_browser)
            hotelarr =[]
            for i in range(3):
                arr = [hotel_info[i] for _  in range(len(reviews[0]))]
                hotelarr.append(arr)
            hotelarr += reviews
            for i in range(7):
                one_page_hotel_reviews[i] += hotelarr[i]
            review_browser.close()
            time.sleep(3)
            url_idx +=1
        except:
            print(url_idx)
            print("끝")
            break
    return one_page_hotel_reviews
    

In [7]:
#호텔 페이지 내에서 각 호텔들 접근하기
def page_look_it_up(browser):
    while True:
        time.sleep(3)
        nums2find_lastPage=len(browser.find_element_by_css_selector("div.goodsListView > div > span").text)
        page_idx =1
        while page_idx <nums2find_lastPage//2 + 1:
            try:
                time.sleep(2)
                browser.find_element_by_xpath("/html/body/section/div/div/div[2]/div[2]/div[4]/div[2]/div/span/a[{}]".format(page_idx)).click()
                time.sleep(1)
                go_inside_url(browser)
                page_idx +=1 
            except:
                pass
    #접근 끝나면 다음 페이지 누르기
        if nums2find_lastPage < 14:
            return "끝"
        
        while True:
            try:
                
                browser.find_element_by_css_selector("div.goodsListView > div > a.nextBtn").click()
                time.sleep(1)
                break
            except:
                pass

In [11]:
go_inside_url(browser)

들어왔다
hotel 시작
1페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
2페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
3페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
4페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
5페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
6페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
7페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
8페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
9페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
10페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
hotel 시작
1페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
2페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
끝
들어왔다
hotel 시작
1페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
2페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
3페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
4페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6
시작 7
시작 8
시작 9
시작 10
5페이지 시작
시작 1
시작 2
시작 3
시작 4
시작 5
시작 6

[['서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 서울',
  '서머셋팰리스 

In [ ]:
# browser = webdriver.Chrome('/Users/hoon2hooni/usr/bin/chromedriver') #크롬 드라이버 입력
# browser.implicitly_wait(5)
# url = "https://travel.interpark.com/checkinnow/categoryMain/search?type=hotel&regioncode1=42001"
# browser.get(url)
# time.sleep(5)
page_look_it_up(browser)

들어왔다
0
ReviewDate=2021-02
ReviewRating = 10
ReviewTitle = 깨끗하고 조용하고 주차가 편리합니다
ReveiwText= 깨끗하고 조용하고 주차가 편리합니다 [more]
ReviewDate=2021-02
ReviewRating = 10
ReviewTitle = 깨끗하고 조용하고 주차가 편리합니다
ReveiwText= 깨끗하고 조용하고 주차가 편리합니다 [more]
ReviewDate=2021-02
ReviewRating = 10
ReviewTitle = 조아요 
ReveiwText= 잘있다 왔네요 만족스럽습니다  [more]
ReviewDate=2021-02
ReviewRating = 8
ReviewTitle = 시설은 오래된 듯...
ReveiwText= 현대식 시설은 아닙니다ㅡ. [more]
ReviewDate=2021-02
ReviewRating = 8.4
ReviewTitle = 업무용 적합한 레지던스
ReveiwText= 중심가에서 업무 보는데 위치가 좋습니다 [more]
ReviewDate=2021-02
ReviewRating = 8.4
ReviewTitle = 중심가 비지니스 위치 적합
ReveiwText= 가족여행 보다는 비지니스용으로 적합한 레지던스입니다...객실 내부 시설이 좀 오래 되었으나 위치는 업무 보기 좋습니다 [more]
ReviewDate=2021-02
ReviewRating = 10
ReviewTitle = 명절여행
ReveiwText= 두번째방문에,투고박스로예약해서,맛있는식사로더더욱조았네요ㅋㅋ다음방문땐다른메뉴로하려고요~~주변관광도역시조았어요!! [more]
ReviewDate=2021-02
ReviewRating = 9.2
ReviewTitle = 다 좋았는데 한가지..
ReveiwText= 화장실 수압이 너무 심하게 약해요
그거 외엔 친절하시고 깨끗했어요 ㅎㅎ [more]
ReviewDate=2021-02
ReviewRating = 10
ReviewTitle = 예약잘못함
ReveiwTe

In [ ]:
nums2find_lastPage=len(browser.find_element_by_css_selector("div.goodsListView > div > span").text)
if nums2find_lastPage < 14:
    print("마지막 페이지")
else:
    print("아직 아니야")

In [19]:
browser.find_element_by_css_selector("div.section.regionGoodsBlock > div.goodsListView > ul").get_attribute()

In [ ]:
#container > div > div.container > div.colRight > div.section.regionGoodsBlock > div.goodsListView > ul > li:nth-child(1) > div.col.data > div:nth-child(1) > div.titArea > a

In [ ]:
#container > div > div.container > div.colRight > div.section.regionGoodsBlock > div.goodsListView > ul > li:nth-child(1) > div.col.data > div:nth-child(1) > div.titArea > a

In [ ]:
#container > div > div.container > div.colRight > div.section.regionGoodsBlock > div.goodsListView > ul > li:nth-child(2) > div.col.data > div:nth-child(1) > div.titArea > a

In [ ]:
df = pd.DataFrame(review_list)
df.to_csv('../csv_output/Tripadvisor.csv',
          index = False,
          header = ["HotelName", "HotelAddress", "HotelRating", "ReviewDate", "ReviewRating", "ReviewTitle", "ReviewText"])

In [ ]:
#한 페이지내에서 리뷰 긁어오기
def get_reviews(browser): #return pandas dataFrame
    i = 0
    items = browser.find_elements_by_xpath("/html/body/section/div/section/div[1]/div[2]/div")
    item_numbers=len(items[2:])
    while i< 15:
        try:
            browser.find_element_by_css_selector("#review > div.reviewsSearchWrap > div:nth-child(3) > div.conWrap > p").text
            browser.find_element_by_css_selector('div:nth-child({}) > div.conWrap > button'.format(i+3)).click()
            time.sleep(1)
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser') #beautiful soup html.parser활용
            extended_review = soup.find("div", {"class": "item extend"})
            ReviewDate= str(extended_review.select('p.txt > span:nth-child(2)'))\
            .strip('[<span>]</')[:7].replace('.','-')
            
            if not extended_review.find("p", {"class": "scoreNum"}):
                return 'stop'
            ReviewRating = extended_review.find("p", {"class": "scoreNum"}).getText()
            #ReviewTitle
            ReviewTitle = extended_review.find("div", {"class": "titWrap"}).getText()
            #ReviewText 
            ReviewText = extended_review.find("div", {"class": "conWrap"}).getText()
            print(f'ReviewDate={ReviewDate}')
            print(f'ReviewRating = {ReviewRating}')
            print(f'ReviewTitle = {ReviewTitle}')
            print(f'ReveiwText= {ReviewText}')
            i += 1
            
        except:
            
            break
                
    print("끝 다음페이지로 이동할 예정")
            